In [1]:
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
cities_weather = pd.read_csv("city_weather.csv")
cities_weather = cities_weather.rename(columns={"Unnamed: 0": "City_ID"})
cities_weather.head()

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
0,0,Ushuaia,-54.8000,-68.3000,48.20,81.0,20.0,5.75,AR,1.617073e+09
1,2,Thompson,55.7435,-97.8558,6.80,78.0,90.0,27.63,CA,1.617074e+09
2,3,Butaritari,3.0707,172.7902,81.66,78.0,61.0,16.44,KI,1.617074e+09
3,4,Busselton,-33.6500,115.3333,81.00,64.0,20.0,3.00,AU,1.617074e+09
4,6,Mataura,-46.1927,168.8643,64.00,77.0,99.0,3.00,NZ,1.617074e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_weather[['latitude', 'longitude']].astype(float)
humidity = cities_weather['humidity'].astype(float)

fig_layout = {
        'width': '1000px',
        'height': '420px',
        'padding': '3px',
        'border': '1px solid black'
}

fig = gmaps.figure(layout = fig_layout, center=(46.0, -5.0), zoom_level=1.5)


# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_weather['humidity']),
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down cities that fit criteria of:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_weather = cities_weather.loc[(cities_weather['max temp'] > 70.00) & (cities_weather['max temp'] < 80.00) & 
                                   (cities_weather['wind speed'] < 10) & (cities_weather['cloudiness'] == 0)].dropna()

ideal_weather

,City_ID,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
52,59,Koutiala,12.3917,-5.4642,79.52,10.0,0.0,7.43,ML,1.617074e+09
62,70,Flinders,-34.5833,150.8552,73.00,49.0,0.0,6.91,AU,1.617074e+09
77,87,Makkah al Mukarramah,21.4267,39.8261,73.54,54.0,0.0,3.04,SA,1.617073e+09
85,95,Saint-Philippe,-21.3585,55.7679,77.00,69.0,0.0,6.91,RE,1.617074e+09
234,260,Tank,32.2217,70.3793,76.95,16.0,0.0,9.55,PK,1.617074e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather[['city_name', 'country', 'latitude', 'longitude']]
# hotel_df = hotel_df.rename(columns={"city_name": "City", "country": "Country", "latitude": "Lat", "longitude": "Lng"})
# hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))
hotel_df['Hotel Name'] = ""
hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(['index'],axis=1)
hotel_df.head()

<ipython-input-5-fd71c46a5243>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""
<ipython-input-5-fd71c46a5243>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Coordinates'] = list(zip(hotel_df['latitude'], hotel_df['longitude']))


,city_name,country,latitude,longitude,Hotel Name,Coordinates
0,Koutiala,ML,12.3917,-5.4642,,"(12.3917, -5.4642)"
1,Flinders,AU,-34.5833,150.8552,,"(-34.5833, 150.8552)"
2,Makkah al Mukarramah,SA,21.4267,39.8261,,"(21.4267, 39.8261)"
3,Saint-Philippe,RE,-21.3585,55.7679,,"(-21.3585, 55.7679)"
4,Tank,PK,32.2217,70.3793,,"(32.2217, 70.3793)"


In [6]:
#convert coordinates columns to a string and remove parathesis
hotel_df['Coordinates'] = hotel_df['Coordinates'].astype(str)
hotel_df['Coordinates'] = hotel_df['Coordinates'].str.replace("(", "")
hotel_df['Coordinates'] = hotel_df['Coordinates'].str.replace(")", "")

In [7]:
# Set parameters to search for a hotel
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    params["location"] = hotel_df['Coordinates'][index]

    # make request and print url and convert to json
    response = requests.get(url, params=params).json()
#     pprint(response)
    
    #Grab the first hotel from the results and store the name in the hotel_df dataframe
    #get first hotel names and which cities had no hotels within radius
    try:
        print(response['results'][0]['name'])
        hotel_df['Hotel Name'][index] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df['Hotel Name'][index] = ""

Hotel Campement Poule Verte


<ipython-input-7-c8eac73e0d74>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][index] = response['results'][0]['name']


Shellharbour Resort & Conference Centre
Makkah Clock Royal Tower, A Fairmont Hotel
Chambres d'hôte "La Trinité"
Saraikistan house


In [8]:
hotel_df

,city_name,country,latitude,longitude,Hotel Name,Coordinates
0,Koutiala,ML,12.3917,-5.4642,Hotel Campement Poule Verte,"12.3917, -5.4642"
1,Flinders,AU,-34.5833,150.8552,Shellharbour Resort & Conference Centre,"-34.5833, 150.8552"
2,Makkah al Mukarramah,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel","21.4267, 39.8261"
3,Saint-Philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité""","-21.3585, 55.7679"
4,Tank,PK,32.2217,70.3793,Saraikistan house,"32.2217, 70.3793"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# template = gmaps.marker_layer(hotel_info)

# Display figure
fig.add_layer(markers)
# fig.add_layer(template)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px', width='1000px'))